<a href="https://colab.research.google.com/github/Sudharshan-Saravanakumar/football-match-predictor/blob/main/PL_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
matches = pd.read_csv('matches.csv', index_col=0)

In [ ]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [ ]:
matches["date"] = pd.to_datetime(matches["date"])

In [ ]:
matches["h/a"] = matches["venue"].astype("category").cat.codes #converting venue to a home (1) or away (0) number

In [ ]:
matches["opp"] = matches["opponent"].astype("category").cat.codes # converting opponents to a number

In [ ]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int") # converting hours to number in case a team plays better at a certain time

In [ ]:
matches["day"] = matches["date"].dt.dayofweek # converting day of week of game to a number

In [ ]:
matches["target"] = (matches["result"] == "W").astype("int") # setting a win to the value 1

In [ ]:
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,dist,fk,pk,pkatt,season,team,h/a,opp,hour,target
1,2021-08-15,16:30,Premier League,Matchweek 1,6,Away,L,0.0,1.0,Tottenham,...,16.9,1.0,0.0,0.0,2022,Manchester City,0,18,16,0
2,2021-08-21,15:00,Premier League,Matchweek 2,5,Home,W,5.0,0.0,Norwich City,...,17.3,1.0,0.0,0.0,2022,Manchester City,1,15,15,1
3,2021-08-28,12:30,Premier League,Matchweek 3,5,Home,W,5.0,0.0,Arsenal,...,14.3,0.0,0.0,0.0,2022,Manchester City,1,0,12,1
4,2021-09-11,15:00,Premier League,Matchweek 4,5,Away,W,1.0,0.0,Leicester City,...,14.0,0.0,0.0,0.0,2022,Manchester City,0,10,15,1
6,2021-09-18,15:00,Premier League,Matchweek 5,5,Home,D,0.0,0.0,Southampton,...,15.7,1.0,0.0,0.0,2022,Manchester City,1,17,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,6,Away,L,0.0,4.0,Tottenham,...,17.4,0.0,0.0,0.0,2021,Sheffield United,0,18,19,0
39,2021-05-08,15:00,Premier League,Matchweek 35,5,Home,L,0.0,2.0,Crystal Palace,...,11.4,1.0,0.0,0.0,2021,Sheffield United,1,6,15,0
40,2021-05-16,19:00,Premier League,Matchweek 36,6,Away,W,1.0,0.0,Everton,...,17.0,0.0,0.0,0.0,2021,Sheffield United,0,7,19,1
41,2021-05-19,18:00,Premier League,Matchweek 37,2,Away,L,0.0,1.0,Newcastle Utd,...,16.0,1.0,0.0,0.0,2021,Sheffield United,0,14,18,0


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators = 100, min_samples_split=10, random_state=1)

In [ ]:
train = matches[matches["date"] < '2022-01-01']

In [ ]:
test = matches[matches["date"] > '2022-01-01']

In [ ]:
predictors = ["h/a", "opp", "hour", "day"]

In [ ]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, random_state=1)

In [ ]:
preds = rf.predict(test[predictors]) #making prediction

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc = accuracy_score(test["target"], preds) #testing accuracy

In [ ]:
acc

0.6123188405797102

In [ ]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [ ]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,140,32
1,75,29


In [ ]:
from sklearn.metrics import precision_score
precision_score(test["target"], preds)

0.47540983606557374

In [ ]:
grouped_matches = matches.groupby("team")

In [ ]:
group = grouped_matches.get_group("Manchester United").sort_values("date")

In [ ]:
def rolling_averages(group, cols, new_cols): # function to take into consideration form of a team
    group = group.sort_values("date") # sorting games by date
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols) # droping missing values and replacing with empty
    return group

In [ ]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols] # creating new columns with rolling average values

rolling_averages(group, cols, new_cols) # calling function and generating average of last 3 games

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,hour,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
5,2020-10-17,20:00,Premier League,Matchweek 5,5,Away,W,4.0,1.0,Newcastle Utd,...,20,1,1.666667,3.666667,9.000000,2.333333,19.800000,0.333333,0.666667,0.666667
7,2020-10-24,17:30,Premier League,Matchweek 6,5,Home,D,0.0,0.0,Chelsea,...,17,0,2.666667,3.000000,12.333333,4.666667,20.000000,0.000000,0.666667,1.000000
9,2020-11-01,16:30,Premier League,Matchweek 7,6,Home,L,0.0,1.0,Arsenal,...,16,0,1.666667,2.333333,15.000000,5.333333,19.700000,0.000000,0.333333,0.666667
11,2020-11-07,12:30,Premier League,Matchweek 8,5,Away,W,3.0,1.0,Everton,...,12,1,1.333333,0.666667,16.666667,5.666667,18.300000,0.000000,0.000000,0.333333
12,2020-11-21,20:00,Premier League,Matchweek 9,5,Home,W,1.0,0.0,West Brom,...,20,1,1.000000,0.666667,12.000000,3.666667,18.733333,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,2022-04-02,17:30,Premier League,Matchweek 31,5,Home,D,1.0,1.0,Leicester City,...,17,0,1.333333,2.000000,12.666667,3.666667,15.333333,0.333333,0.000000,0.000000
41,2022-04-09,12:30,Premier League,Matchweek 32,5,Away,L,0.0,1.0,Everton,...,12,0,1.666667,2.333333,9.000000,4.333333,14.333333,0.000000,0.000000,0.000000
42,2022-04-16,15:00,Premier League,Matchweek 33,5,Home,W,3.0,2.0,Norwich City,...,15,1,1.333333,1.333333,11.333333,5.000000,15.333333,0.000000,0.000000,0.000000
43,2022-04-19,20:00,Premier League,Matchweek 30,1,Away,L,0.0,4.0,Liverpool,...,20,0,1.333333,1.333333,14.333333,6.000000,15.666667,0.333333,0.000000,0.000000


In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [ ]:
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 6  2020-10-04  14:00  Premier League   Matchweek 4   
                        7  2020-10-17  17:30  Premier League   Matchweek 5   
                        9  2020-10-25  19:15  Premier League   Matchweek 6   
                        11 2020-11-01  16:30  Premier League   Matchweek 7   
                        13 2020-11-08  19:15  Premier League   Matchweek 8   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 32 2022-03-13  14:00  Premier League  Matchweek 29   
                        33 2022-03-18  20:00  Premier League  Matchweek 30   
                        34 2022-04-02  15:00  Premier League  Matchweek 31   
                        35 2022-04-08  20:00  Premier League  Matchweek 32   
                        36 2022-04-24  14:00  Premier League  Matchweek 34   

                            day venue result   gf   ga         opponent  ...  \
team                                                                     ...   
Arsenal                 6     6  Home      W  2.0  1.0    Sheffield Utd  ...   
                        7     5  Away      L  0.0  1.0  Manchester City  ...   
                        9     6  Home      L  0.0  1.0   Leicester City  ...   
                        11    6  Away      W  1.0  0.0   Manchester Utd  ...   
                        13    6  Home      L  0.0  3.0      Aston Villa  ...   
...                         ...   ...    ...  ...  ...              ...  ...   
Wolverhampton Wanderers 32    6  Away      W  1.0  0.0          Everton  ...   
                        33    4  Home      L  2.0  3.0     Leeds United  ...   
                        34    5  Home      W  2.0  1.0      Aston Villa  ...   
                        35    4  Away      L  0.0  1.0    Newcastle Utd  ...   
                        36    6  Away      L  0.0  1.0          Burnley  ...   

                            hour  target  gf_rolling  ga_rolling sh_rolling  \
team                                                                          
Arsenal                 6     14       1    2.000000    1.333333   7.666667   
                        7     17       0    1.666667    1.666667   5.333333   
                        9     19       0    1.000000    1.666667   7.000000   
                        11    16       1    0.666667    1.000000   9.666667   
                        13    19       0    0.333333    0.666667   9.666667   
...                          ...     ...         ...         ...        ...   
Wolverhampton Wanderers 32    14       1    1.333333    1.000000  12.333333   
                        33    20       0    1.666667    0.666667  12.333333   
                        34    15       1    2.333333    1.000000  13.000000   
                        35    20       0    1.666667    1.333333  13.000000   
                        36    14       0    1.333333    1.666667  10.000000   

                           sot_rolling dist_rolling fk_rolling  pk_rolling  \
team                                                                         
Arsenal                 6     3.666667    14.733333   0.666667    0.000000   
                        7     3.666667    15.766667   0.000000    0.000000   
                        9     3.666667    16.733333   0.666667    0.000000   
                        11    4.000000    16.033333   1.000000    0.000000   
                        13    2.666667    18.033333   1.000000    0.333333   
...                                ...          ...        ...         ...   
Wolverhampton Wanderers 32    3.666667    19.300000   0.000000    0.000000   
                        33    4.333333    19.600000   0.000000    0.000000   
                        34    5.333333    19.833333   0.000000    0.000000   
                        35    5.000000    18.533333   0.000000    0.000000   
             

In [ ]:
matches_rolling = matches_rolling.droplevel('team') # dropping extra index level

In [ ]:
matches_rolling.index = range(matches_rolling.shape[0]) # adding new index

In [ ]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,hour,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2020-10-04,14:00,Premier League,Matchweek 4,6,Home,W,2.0,1.0,Sheffield Utd,...,14,1,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000
1,2020-10-17,17:30,Premier League,Matchweek 5,5,Away,L,0.0,1.0,Manchester City,...,17,0,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000
2,2020-10-25,19:15,Premier League,Matchweek 6,6,Home,L,0.0,1.0,Leicester City,...,19,0,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000
3,2020-11-01,16:30,Premier League,Matchweek 7,6,Away,W,1.0,0.0,Manchester Utd,...,16,1,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000
4,2020-11-08,19:15,Premier League,Matchweek 8,6,Home,L,0.0,3.0,Aston Villa,...,19,0,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,2022-03-13,14:00,Premier League,Matchweek 29,6,Away,W,1.0,0.0,Everton,...,14,1,1.333333,1.000000,12.333333,3.666667,19.300000,0.000000,0.000000,0.000000
1313,2022-03-18,20:00,Premier League,Matchweek 30,4,Home,L,2.0,3.0,Leeds United,...,20,0,1.666667,0.666667,12.333333,4.333333,19.600000,0.000000,0.000000,0.000000
1314,2022-04-02,15:00,Premier League,Matchweek 31,5,Home,W,2.0,1.0,Aston Villa,...,15,1,2.333333,1.000000,13.000000,5.333333,19.833333,0.000000,0.000000,0.000000
1315,2022-04-08,20:00,Premier League,Matchweek 32,4,Away,L,0.0,1.0,Newcastle Utd,...,20,0,1.666667,1.333333,13.000000,5.000000,18.533333,0.000000,0.000000,0.000000


In [ ]:
def make_predictions(data, predictors): # making the predictions
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors]) # making prediction
    combined = pd.DataFrame(dict(actual=test["target"], prediction=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision # returning the values for the prediction

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
precision

0.6458333333333334

In [ ]:
combined

,actual,prediction
55,0,1
56,1,0
57,1,0
58,1,1
59,1,1
...,...,...
1312,1,0
1313,0,0
1314,1,0
1315,0,1


In [ ]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index = True, right_index = True)

In [ ]:
combined.head(10)

,actual,prediction,date,team,opponent,result
55,0,1,2022-01-23,Arsenal,Burnley,D
56,1,0,2022-02-10,Arsenal,Wolves,W
57,1,0,2022-02-19,Arsenal,Brentford,W
58,1,1,2022-02-24,Arsenal,Wolves,W
59,1,1,2022-03-06,Arsenal,Watford,W
60,1,1,2022-03-13,Arsenal,Leicester City,W
61,0,0,2022-03-16,Arsenal,Liverpool,L
62,1,0,2022-03-19,Arsenal,Aston Villa,W
63,0,0,2022-04-04,Arsenal,Crystal Palace,L
64,0,0,2022-04-09,Arsenal,Brighton,L


In [ ]:
class MissingDict(dict): ## creating a class that inherits from the dictionary class
    __missing__ = lambda self, key: key ## in case a team name is missing

map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}
mapping = MissingDict(**map_values)

In [ ]:
mapping["West Ham United"]

'West Ham'

In [ ]:
combined["new_team"] = combined["team"].map(mapping)

In [ ]:
combined

,actual,prediction,date,team,opponent,result,new_team
55,0,1,2022-01-23,Arsenal,Burnley,D,Arsenal
56,1,0,2022-02-10,Arsenal,Wolves,W,Arsenal
57,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal
58,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal
59,1,1,2022-03-06,Arsenal,Watford,W,Arsenal
...,...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves
1313,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves
1314,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves
1315,0,1,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves


In [ ]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"]) # finding both the home and away team predictions and merging them


In [ ]:
merged

,actual_x,prediction_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,prediction_y,team_y,opponent_y,result_y,new_team_y
0,0,1,2022-01-23,Arsenal,Burnley,D,Arsenal,0,0,Burnley,Arsenal,D,Burnley
1,1,0,2022-02-10,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal,0,0,Brentford,Arsenal,L,Brentford
3,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,Arsenal,0,0,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves,0,0,Everton,Wolves,L,Everton
243,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves,1,0,Leeds United,Wolves,W,Leeds United
244,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves,0,0,Aston Villa,Wolves,L,Aston Villa
245,0,1,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle United


In [ ]:
merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] ==0)]["actual_x"].value_counts()

,count
actual_x,
1,22
0,10
